<a href="https://colab.research.google.com/github/mingmingbupt/tensorflow/blob/master/tf_data_generate_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0-rc2
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.2
pandas 1.0.3
sklearn 0.22.2.post1
tensorflow 2.2.0-rc2
tensorflow.keras 2.3.0-tf


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

In [3]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state = 7)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state = 11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)


(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [5]:
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [0]:
#我们把scale后的数据呢，生产csv文件
output_dir = "generate_csv" #定义一个文件夹，把生成的文件放到这个文件夹下
if not os.path.exists(output_dir): #如果不存在文件夹的话，就用os.mkdir去创建这个文件夹
    os.mkdir(output_dir)

def save_to_csv(output_dir, data, name_prefix,
                header=None, n_parts=10): #把一个单独的dataset,train/valid/test,保存到文件当中去
                # output_dir： 输出文件夹，
                # data ： dataset
                # name_prefix： 因为我们要分别给train/valid/test分别生成csv文件，所以需要name_prefix去做区分
                # header： 对于csv文件呢，可能需要一个header,这里默认是None
                # n_parts: 我们的datesets是可以把多个文件去merge起来形成一个数据集的，
                #      所以说呢，为了测试这个场景，我们需要把我们的数据集去切分成多个文件来进行存储，这里默认是10个文件
    path_format = os.path.join(output_dir, "{}_{:02d}.csv") # 生成文件名 第一个{}里面天train,test还是valid. 第二个中括号填的是两位的一个整数
    # 如果是一位的一个整数，就需要补全一位，这里填每个n_parts的具体数字
    filenames = [] #最后会返回所有的文件名
    
    # enumerate就是给每一组标记一个值，也就是说每一组呢，我们可以通过这个row_indices获得，它标记的值呢，就是file_idx
    # 就是一个整数，这个整数可以用来当成一个file_id
    for file_idx, row_indices in enumerate(
        #我用一个np.arange生成了一个和data一样长度的数组，也就是说我data里面有n个元素，那我们这个数组里面也有n个元素
        #它的元素值就是0到n-1，用这个数值当索引。我们再把当索引的这个数值，分成了n_parts个部分
        #有了这n_parts部分以后，用每一组里面的索引去data里面去取元素，从而获得数据
        np.array_split(np.arange(len(data)), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx) #生成对应的子文件名
        filenames.append(part_csv)
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header + "\n")
            for row_index in row_indices: #行索引
                f.write(",".join(
                    [repr(col) for col in data[row_index]])) #用逗号分割，需要做字符串化处理
                f.write('\n')
    return filenames

train_data = np.c_[x_train_scaled, y_train]  # np.c_把数据按行merge
valid_data = np.c_[x_valid_scaled, y_valid]  
test_data = np.c_[x_test_scaled, y_test]
# 这样就把train valid test数据merge起来了
# 然后生成header,从housing数据集获得，因为我们把y也拼接起来了，所以也要加个y的名字MidianHouseValue
header_cols = housing.feature_names + ["MidianHouseValue"]
header_str = ",".join(header_cols) # 用逗号连接起来

train_filenames = save_to_csv(output_dir, train_data, "train",
                              header_str, n_parts=20) 
valid_filenames = save_to_csv(output_dir, valid_data, "valid",
                              header_str, n_parts=10)
test_filenames = save_to_csv(output_dir, test_data, "test",
                             header_str, n_parts=10)

In [10]:
import pprint
print("train filenames:")
pprint.pprint(train_filenames)
print("valid filenames:")
pprint.pprint(valid_filenames)
print("test filenames:")
pprint.pprint(test_filenames)

train filenames:
['generate_csv/train_00.csv',
 'generate_csv/train_01.csv',
 'generate_csv/train_02.csv',
 'generate_csv/train_03.csv',
 'generate_csv/train_04.csv',
 'generate_csv/train_05.csv',
 'generate_csv/train_06.csv',
 'generate_csv/train_07.csv',
 'generate_csv/train_08.csv',
 'generate_csv/train_09.csv',
 'generate_csv/train_10.csv',
 'generate_csv/train_11.csv',
 'generate_csv/train_12.csv',
 'generate_csv/train_13.csv',
 'generate_csv/train_14.csv',
 'generate_csv/train_15.csv',
 'generate_csv/train_16.csv',
 'generate_csv/train_17.csv',
 'generate_csv/train_18.csv',
 'generate_csv/train_19.csv']
valid filenames:
['generate_csv/valid_00.csv',
 'generate_csv/valid_01.csv',
 'generate_csv/valid_02.csv',
 'generate_csv/valid_03.csv',
 'generate_csv/valid_04.csv',
 'generate_csv/valid_05.csv',
 'generate_csv/valid_06.csv',
 'generate_csv/valid_07.csv',
 'generate_csv/valid_08.csv',
 'generate_csv/valid_09.csv']
test filenames:
['generate_csv/test_00.csv',
 'generate_csv/test_0

In [11]:
# 1. filename -> dataset
# 2. read file -> dataset -> datasets -> merge
# 3. parse csv
# tensorflow将csv内容读取出来形成dataset。分两步
# 1. 把这些filenames生成一个dataset
# 2. 对于这个filename组对应的dataset组里面的每一个元素，我们去读取文件，生成一个dataset,
#   这样我们就得到多个dataset,也就是dataset组，然后我再把这个dataset组merge起来
#   形成一个总的dataset
# 其中第一步呢，我们生成一个dataset就可以啦
# 第二步呢，我们使用interleave api去进行操作

#文件名数据集 这是第一步
filename_dataset = tf.data.Dataset.list_files(train_filenames) #list_files是专门处理文件名的，把文件名生成一个dataset
for filename in filename_dataset:
    print(filename)

tf.Tensor(b'generate_csv/train_18.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_17.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_13.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_15.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_05.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_14.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_02.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_19.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_11.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_12.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_06.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_09.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_03.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_08.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_00.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_07.csv', 

In [12]:
# 第二步，这一步我们使用interleave的api,这个api呢，是dataset的一个函数，这个函数用来遍历dataset里面的每一个元素
# 对每一个元素进行操作，最后将操作完的结果合并起来
# 第一个参数 依然是一个函数
# 第二个参数 
n_readers = 5
#tf.data.TextLineDataset这个是专门读取文本文件的api，这个api呢，可以把一个文件中对应的文本内容读取出来，生成一个dataset
#也就是按行读取文本，生成dataset
dataset = filename_dataset.interleave(
    lambda filename: tf.data.TextLineDataset(filename).skip(1), # 对每一个filename, 都需要构建一个数据集出来，并用skip(1)把header省略掉
    cycle_length = n_readers # 用来控制读取文件的并行度的
)
#这样我们就把文件集合中的所有文件都读取出来，并将文件内容汇合成一个大的dataset
for line in dataset.take(15): #我们看下这个dataset文件内容是什么
    print(line.numpy())

# 这样我们就得到一个大的dataset,这个大的dataset里面的内容呢，一行就是一个样本的文本
# 我们需要把他解析出来

b'0.04326300977263167,-1.0895425985107923,-0.38878716774583305,-0.10789864528874438,-0.6818663605100649,-0.0723871014747467,-0.8883662012710817,0.8213992340186296,1.426'
b'-0.8219588176953616,1.874166156711919,0.18212349433218608,-0.03170019246279883,-0.6011178900722581,-0.14337494105109344,1.0852205298015787,-0.8613994495208361,1.054'
b'0.42408210084996534,0.9129633171802288,-0.04437481876046234,-0.15297213746739335,-0.24727627804141977,-0.10539166599677323,0.8612674255663844,-1.3357789003702432,3.955'
b'0.6303435674178064,1.874166156711919,-0.06713214279531016,-0.12543366804152128,-0.19737553788322462,-0.022722631725889016,-0.692407235065288,0.7265233438487496,2.419'
b'0.09734603446040174,0.7527628439249472,-0.20218964416999152,-0.1954700015215477,-0.4060513603629498,0.006785531677655949,-0.813715166526018,0.656614793197258,1.119'
b'-0.7543417158936074,-0.9293421252555106,-0.9212720434835953,0.1242806741969112,-0.5983960315181748,-0.18494335623235414,-0.8183808561975836,0.85136004144

In [15]:
# tf.io.decode_csv(str, record_defaults)
# 我们需要把他解析出来，如何解析呢，需要用到tf.io.decode_csv(str, record_defaults)
# 这个api第一个参数就是字符串str, 第二个字符串就是record_defaults，用来定义字符串各个field的类型是什么
sample_str = '1,2,3,4,5'
# record_defaults = [tf.constant(0, dtype=tf.int32)] * 5
record_defaults = [#定义的是这5个元素分别的类型，以及他们的默认值，也就是当值有确实的时候，应该设置为什么
          #因为是5列，所以record_defaults也是五个元素
    tf.constant(0, dtype=tf.int32),  #第一个是tf.contant
    0, #可以通过普通的类型，他会自动转换
    np.nan,
    "hello",
    tf.constant([]) #无固定类型，默认是float32
]

#其实1还可以被表示为int float string 等等，
parsed_fields = tf.io.decode_csv(sample_str, record_defaults)
print(parsed_fields)
#

[<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: shape=(), dtype=string, numpy=b'4'>, <tf.Tensor: shape=(), dtype=float32, numpy=5.0>]


In [16]:
try:
    parsed_fields = tf.io.decode_csv(',,,,', record_defaults) #如果传进来的字符串是一个不对字符串，会报什么错
except tf.errors.InvalidArgumentError as ex:
    print(ex)

Field 4 is required but missing in record 0! [Op:DecodeCSV]


In [17]:
try:
    parsed_fields = tf.io.decode_csv('1,2,3,4,5,6,7', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

Expect 5 fields but have 7 in record 0 [Op:DecodeCSV]


In [18]:
#把csv的一行解析出来，第一个参数就是输入的str,第二个参数就是要解析出来多少个field
def parse_csv_line(line, n_fields = 9):
    defs = [tf.constant(np.nan)] * n_fields #就是我们的record_defaults，是具有n_fields的float值，
    parsed_fields = tf.io.decode_csv(line, record_defaults=defs) # 用tf.io.decode_csv把字符串解析出来
    x = tf.stack(parsed_fields[0:-1]) # 每一行里有n_fields=9个元素，其中前8个元素是x，最后一个元素是y
    y = tf.stack(parsed_fields[-1:]) # 我需要把前8个值变成一个向量，第9个值变成一个向量。变成向量需要tf.stack进行转换
    return x, y

parse_csv_line(b'-0.9868720801669367,0.832863080552588,-0.18684708416901633,-0.14888949288707784,-0.4532302419670616,-0.11504995754593579,1.6730974284189664,-0.7465496877362412,1.138',
               n_fields=9)

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([-0.9868721 ,  0.8328631 , -0.18684709, -0.1488895 , -0.45323023,
        -0.11504996,  1.6730974 , -0.74654967], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.138], dtype=float32)>)

In [20]:
# 1. filename -> dataset
# 2. read file -> dataset -> datasets -> merge
# 3. parse csv
# 将csv文件读取出来，形成dataset完整过程
# filenames 读取文件名
# n_readers 并行程度
# n_parse_threads 我们在parses时候呢，并行程度是多少，是用来做解析的时候，并行度是多少
# shuffle_buffer_size shuffle的时候我们的buffer是多少，这里设成10000，是我用来混排数据的时候，我的buffer的size是多大
def csv_reader_dataset(filenames, n_readers=5,
                       batch_size=32, n_parse_threads=5,
                       shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames) #1.把filename变成一个dataset
    dataset = dataset.repeat() #做repeat,数据集重复多少次，如果不加参数，就是重复无限次
    dataset = dataset.interleave( #文件名转换成文本内容
        lambda filename: tf.data.TextLineDataset(filename).skip(1), #忽略header第一行
        cycle_length = n_readers
    )
    dataset.shuffle(shuffle_buffer_size) #做shuffle,即我把数据做混排，shuffle需要shuffle_buffer_size，即shuffle需要的内存有多大
    dataset = dataset.map(parse_csv_line, #将dataset每一个元素做个操作，将返回的结果组成一个新的dataset
                          num_parallel_calls=n_parse_threads) #num_parallel_calls 并行程度
    #我们发现map和interleave做的事情很相似，interleave做的事情是对于原来dataset里面的每一个元素，都会生成一个新的dataset,再把这些
    #新的dataset合并，生成一个大的dataset。
    dataset = dataset.batch(batch_size) #做batch操作
    return dataset #返回dataset

train_set = csv_reader_dataset(train_filenames, batch_size=3)
for x_batch, y_batch in train_set.take(2):
    print("x:")
    pprint.pprint(x_batch)
    print("y:")
    pprint.pprint(y_batch)

x:
<tf.Tensor: shape=(3, 8), dtype=float32, numpy=
array([[ 0.48530516, -0.8492419 , -0.06530126, -0.02337966,  1.4974351 ,
        -0.07790658, -0.90236324,  0.78145146],
       [-1.119975  , -1.3298433 ,  0.14190045,  0.4658137 , -0.10301778,
        -0.10744184, -0.7950524 ,  1.5304717 ],
       [ 0.63034356,  1.8741661 , -0.06713215, -0.12543367, -0.19737554,
        -0.02272263, -0.69240725,  0.72652334]], dtype=float32)>
y:
<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[2.956],
       [0.66 ],
       [2.419]], dtype=float32)>
x:
<tf.Tensor: shape=(3, 8), dtype=float32, numpy=
array([[ 0.09734604,  0.75276285, -0.20218964, -0.19547   , -0.40605137,
         0.00678553, -0.81371516,  0.6566148 ],
       [-0.66722274, -0.04823952,  0.34529406,  0.53826684,  1.8521839 ,
        -0.06112538, -0.8417093 ,  1.5204847 ],
       [-0.7432054 ,  0.91296333, -0.64432025, -0.1479097 ,  0.7398511 ,
         0.11427691, -0.7950524 ,  0.68158215]], dtype=float32)>
y:
<tf.Tensor: shape=(

In [0]:
batch_size = 32
train_set = csv_reader_dataset(train_filenames,
                               batch_size = batch_size)
valid_set = csv_reader_dataset(valid_filenames,
                               batch_size = batch_size)
test_set = csv_reader_dataset(test_filenames,
                              batch_size = batch_size)
#训练集 测试集 验证集都取出来

In [23]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu',
                       input_shape=[8]),
    keras.layers.Dense(1),
])
model.compile(loss="mean_squared_error", optimizer="sgd")
callbacks = [keras.callbacks.EarlyStopping(
    patience=5, min_delta=1e-2)]

#x和y都在train_set里
history = model.fit(train_set,
                    validation_data = valid_set,
                    steps_per_epoch = 11160 // batch_size, #因为我们的dataset是不停的去产生数据的，所以说我们的fit函数不知道跑多少次
                    #这里就用11160（就是我们train_set的大小），除以batch_size呢，就是遍历一次需要多少次迭代
                    validation_steps = 3870 // batch_size, #同理我的valid data也需要这样一个值
                    epochs = 100,
                    callbacks = callbacks)

Epoch 1/100
348/348 [==============================] - 1s 3ms/step - loss: 1.6179 - val_loss: 5.1179
Epoch 2/100
348/348 [==============================] - 1s 2ms/step - loss: 0.5217 - val_loss: 0.4282
Epoch 3/100
348/348 [==============================] - 1s 2ms/step - loss: 0.4083 - val_loss: 0.3994
Epoch 4/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3815 - val_loss: 0.3828
Epoch 5/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3711 - val_loss: 0.3848
Epoch 6/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3588 - val_loss: 0.3776
Epoch 7/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3828 - val_loss: 0.3825
Epoch 8/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3644 - val_loss: 0.3730
Epoch 9/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3521 - val_loss: 0.3672
Epoch 10/100
348/348 [==============================] - 1s 2ms/step - loss: 0.3613 - val_lo

In [24]:
model.evaluate(test_set, steps = 5160 // batch_size) #因为dataset是不停产生数据集的，所以也需要指定step,用test_size=5160,除以batch_size

161/161 [==============================] - 0s 2ms/step - loss: 0.3664


0.3664287328720093